In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import os

In [21]:
# %load_ext cudf.pandas

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


In [22]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
import polars as pl
from xgboost import XGBClassifier
import gc

In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Print filepaths and save to file_dirs 
file_dirs = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file_dirs.append(dirname + '/' + filename)
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [24]:
train = pl.read_csv(file_dirs[1])
target = pl.read_csv(file_dirs[2])

In [25]:
train = train.drop("id")

In [26]:
target = target.drop("id")

In [27]:
category_columns = [col for col, dtype in zip(train.columns, train.dtypes) if dtype == pl.Utf8]

In [28]:
target = target.with_columns(pl.lit(0).cast(pl.Int64).alias('Response'))

In [29]:
df = pl.concat([train, target])

In [30]:
df

Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
str,i64,i64,f64,i64,str,str,f64,f64,i64,i64
"""Male""",21,1,35.0,0,"""1-2 Year""","""Yes""",65101.0,124.0,187,0
"""Male""",43,1,28.0,0,"""> 2 Years""","""Yes""",58911.0,26.0,288,1
"""Female""",25,1,14.0,1,"""< 1 Year""","""No""",38043.0,152.0,254,0
"""Female""",35,1,1.0,0,"""1-2 Year""","""Yes""",2630.0,156.0,76,0
"""Female""",36,1,15.0,1,"""1-2 Year""","""No""",31951.0,152.0,294,0
…,…,…,…,…,…,…,…,…,…,…
"""Male""",57,1,28.0,0,"""1-2 Year""","""Yes""",51661.0,124.0,109,0
"""Male""",28,1,50.0,1,"""< 1 Year""","""No""",25651.0,152.0,184,0
"""Male""",47,1,33.0,1,"""1-2 Year""","""No""",2630.0,138.0,63,0


In [31]:
df = df.with_columns([
    pl.col('Region_Code').cast(int),
    pl.col('Annual_Premium').cast(int),
    pl.col('Policy_Sales_Channel').cast(int)
])

In [35]:
df

Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
str,i64,i64,i64,i64,str,str,i64,i64,i64,i64
"""Male""",21,1,35,0,"""1-2 Year""","""Yes""",65101,124,187,0
"""Male""",43,1,28,0,"""> 2 Years""","""Yes""",58911,26,288,1
"""Female""",25,1,14,1,"""< 1 Year""","""No""",38043,152,254,0
"""Female""",35,1,1,0,"""1-2 Year""","""Yes""",2630,156,76,0
"""Female""",36,1,15,1,"""1-2 Year""","""No""",31951,152,294,0
…,…,…,…,…,…,…,…,…,…,…
"""Male""",57,1,28,0,"""1-2 Year""","""Yes""",51661,124,109,0
"""Male""",28,1,50,1,"""< 1 Year""","""No""",25651,152,184,0
"""Male""",47,1,33,1,"""1-2 Year""","""No""",2630,138,63,0


In [37]:
df_pd = df.to_pandas()

In [38]:
df = df_pd

In [41]:
df

KeyboardInterrupt: 

In [ ]:
# df = df.with_columns([
#     (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Annual_Premium'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Annual_Premium'),
#     (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vehicle_Age'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vehicle_Age'),
#     (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vehicle_Damage'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vehicle_Damage'),
#     (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vintage'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vintage')
# ])

In [42]:
df_pd['Previously_Insured_Annual_Premium'] = pd.factorize(df_pd['Previously_Insured'].astype(str) + df_pd['Annual_Premium'].astype(str))[0]
df_pd['Previously_Insured_Vehicle_Age'] = pd.factorize(df_pd['Previously_Insured'].astype(str) + df_pd['Vehicle_Age'].astype(str))[0]
df_pd['Previously_Insured_Vehicle_Damage'] = pd.factorize(df_pd['Previously_Insured'].astype(str) + df_pd['Vehicle_Damage'].astype(str))[0]
df_pd['Previously_Insured_Vintage'] = pd.factorize(df_pd['Previously_Insured'].astype(str) + df_pd['Vintage'].astype(str))[0]

In [46]:
df

KeyboardInterrupt: 

In [43]:
tar='Response'

In [44]:
train = df[:train.shape[0]]
test = df[train.shape[0]:]

In [45]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [47]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [48]:
train =reduce_mem_usage(train)
test = reduce_mem_usage(test)

Memory usage of dataframe is 1354.60 MB
Memory usage after optimization is: 263.32 MB
Decreased by 80.6%
Memory usage of dataframe is 903.07 MB
Memory usage after optimization is: 175.55 MB
Decreased by 80.6%


In [49]:
initial_features = test.columns
print(initial_features)

Index(['Gender', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured',
       'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium',
       'Policy_Sales_Channel', 'Vintage', 'Response',
       'Previously_Insured_Annual_Premium', 'Previously_Insured_Vehicle_Age',
       'Previously_Insured_Vehicle_Damage', 'Previously_Insured_Vintage'],
      dtype='object')


In [50]:
categorical_features = [col for col in initial_features if pd.concat([train[col], test[col]]).nunique() < 10]
categorical_features.remove('Response')
print(categorical_features)

['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Previously_Insured_Vehicle_Age', 'Previously_Insured_Vehicle_Damage']


In [51]:
numerical_features = list(set(initial_features) - set(categorical_features))

print(numerical_features)

['Policy_Sales_Channel', 'Response', 'Annual_Premium', 'Region_Code', 'Age', 'Previously_Insured_Vintage', 'Previously_Insured_Annual_Premium', 'Vintage']


In [52]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 15 columns):
 #   Column                             Dtype
---  ------                             -----
 0   Gender                             category
 1   Age                                int8
 2   Driving_License                    int8
 3   Region_Code                        int8
 4   Previously_Insured                 int8
 5   Vehicle_Age                        category
 6   Vehicle_Damage                     category
 7   Annual_Premium                     int32
 8   Policy_Sales_Channel               int16
 9   Vintage                            int16
 10  Response                           int8
 11  Previously_Insured_Annual_Premium  int32
 12  Previously_Insured_Vehicle_Age     int8
 13  Previously_Insured_Vehicle_Damage  int8
 14  Previously_Insured_Vintage         int16
dtypes: category(3), int16(3), int32(2), int8(7)
memory usage: 263.3 MB


In [53]:
ddf = train.copy()
x= ddf.drop(columns=['Response'],axis=1)
y= ddf['Response']

In [54]:
test = test.drop(columns=['Response'],axis=1)

In [55]:
x[categorical_features] = x[categorical_features].astype("category")
test[categorical_features] = test[categorical_features].astype("category")

In [56]:
X=x

In [57]:
X.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 14 columns):
 #   Column                             Dtype
---  ------                             -----
 0   Gender                             category
 1   Age                                int8
 2   Driving_License                    category
 3   Region_Code                        int8
 4   Previously_Insured                 category
 5   Vehicle_Age                        category
 6   Vehicle_Damage                     category
 7   Annual_Premium                     int32
 8   Policy_Sales_Channel               int16
 9   Vintage                            int16
 10  Previously_Insured_Annual_Premium  int32
 11  Previously_Insured_Vehicle_Age     category
 12  Previously_Insured_Vehicle_Damage  category
 13  Previously_Insured_Vintage         int16
dtypes: category(7), int16(3), int32(2), int8(2)
memory usage: 252.4 MB


In [ ]:
cat_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'class_names': [0, 1],
    'learning_rate': 0.075,
    'iterations': 3000,
    'depth': 9,
    'random_strength': 0,
    'l2_leaf_reg': 0.5,
    'max_leaves': 512,
    'fold_permutation_block': 64,
    'task_type': 'GPU',
    'random_seed': 42,
    'verbose': False,
    'allow_writing_files': False
}

In [59]:
cat_params = {
    "loss_function":'Logloss',
    "eval_metric":'AUC',
    'learning_rate':0.05,
    'iterations':3000,
    'depth':9,
    'random_strength':0,
    'l2_leaf_reg':0.5,
    'task_type':'GPU',
    'random_seed':42,
    'verbose':False
}

In [60]:
import catboost as cb
from sklearn.model_selection import train_test_split, StratifiedKFold

In [61]:
skfold = StratifiedKFold(5, shuffle=True, random_state=42)

In [62]:
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay, classification_report

In [63]:
cat_params

{'loss_function': 'Logloss',
 'eval_metric': 'AUC',
 'learning_rate': 0.05,
 'iterations': 3000,
 'depth': 9,
 'random_strength': 0,
 'l2_leaf_reg': 0.5,
 'task_type': 'GPU',
 'random_seed': 42,
 'verbose': False}

In [66]:
X.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 14 columns):
 #   Column                             Dtype
---  ------                             -----
 0   Gender                             category
 1   Age                                int8
 2   Driving_License                    category
 3   Region_Code                        int8
 4   Previously_Insured                 category
 5   Vehicle_Age                        category
 6   Vehicle_Damage                     category
 7   Annual_Premium                     int32
 8   Policy_Sales_Channel               int16
 9   Vintage                            int16
 10  Previously_Insured_Annual_Premium  int32
 11  Previously_Insured_Vehicle_Age     category
 12  Previously_Insured_Vehicle_Damage  category
 13  Previously_Insured_Vintage         int16
dtypes: category(7), int16(3), int32(2), int8(2)
memory usage: 252.4 MB


In [67]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 11504798 entries, 0 to 11504797
Series name: Response
Non-Null Count     Dtype
--------------     -----
11504798 non-null  int8 
dtypes: int8(1)
memory usage: 11.0 MB


In [76]:
a=X.columns.values.tolist()

In [88]:
a

['Gender',
 'Age',
 'Driving_License',
 'Region_Code',
 'Previously_Insured',
 'Vehicle_Age',
 'Vehicle_Damage',
 'Annual_Premium',
 'Policy_Sales_Channel',
 'Vintage',
 'Previously_Insured_Annual_Premium',
 'Previously_Insured_Vehicle_Age',
 'Previously_Insured_Vehicle_Damage',
 'Previously_Insured_Vintage']

In [75]:
type(test)

pandas.core.frame.DataFrame

In [83]:
type(test.head().astype(str))

pandas.core.frame.DataFrame

In [86]:
test = pd.DataFrame(test, columns=test.columns)

In [95]:
import cudf

In [ ]:
train = cudf.DataFrame.from_records(train.to_dict('records'))

In [ ]:
test = cudf.DataFrame.from_records(test.to_dict('records'))

In [94]:
oof_preds = []
oof_aucs = []

test_pool = cb.Pool(test.astype(str).to_pandas(), cat_features=a)

for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y[train_idx]
    X_test, y_test = X.iloc[test_idx], y[test_idx]
    
    X_train_pool = cb.Pool(X_train.astype(str), y_train, cat_features=a)
    X_test_pool = cb.Pool(X_test.astype(str), y_test, cat_features=a)
    
    cat_clf = cb.CatBoostClassifier(**cat_params)
    cat_clf = cat_clf.fit(X=X_train_pool,
                          eval_set=X_test_pool,
                          verbose=10,
                          early_stopping_rounds=200)
    
    test_pred = cat_clf.predict_proba(test_pool)[:, 1]
    
    oof_preds.append(test_pred)
    auc = cat_clf.best_score_['validation']['AUC']
    oof_aucs.append(auc)
    print(f"\n---- Fold {fold}: ROC-AUC Score: {auc:.6f}\n")
    
    del X_train, y_train, X_test, y_test
    del X_train_pool, X_test_pool
    del cat_clf
    gc.collect()

auc_mean = np.mean(oof_aucs)
auc_std = np.std(oof_aucs)
print(f"\n---> ROC-AUC Score: {auc_mean:.6f} \xB1 {auc_std:.6f}\n")

test_pred_cat = np.mean(oof_preds, axis=0)

AttributeError: 'DataFrame' object has no attribute 'to_pandas'

In [45]:
submit = pd.read_csv(file_dirs[0])
pred = test_pred_cat
submit["Response"] = pred
submit.to_csv("submission_catboo3.csv", index=False)
submit.head()

,id,Response
0,11504798,0.004384
1,11504799,0.668936
2,11504800,0.245505
3,11504801,0.000096
4,11504802,0.195554


In [37]:
output = pd.DataFrame({
    'id': test.index,
    'Response': test_pred_cat
})

In [38]:
output.to_csv('submission_catboost.csv', index = False)

In [39]:
output.head(5)

,id,Response
0,0,0.004384
1,1,0.668936
2,2,0.245505
3,3,0.000096
4,4,0.195554


In [40]:
output.tail(5)

,id,Response
7669861,7669861,0.195308
7669862,7669862,0.000169
7669863,7669863,0.000531
7669864,7669864,0.592451
7669865,7669865,0.000087


In [41]:
len(output)

7669866

In [43]:
output.to_csv('/kaggle/working/submission_catboost_test.csv', index = False)